In [59]:
!pip install -q sql av pillow pandas sqlalchemy ipython-sql pymysql

In [60]:
import pandas as pd
import pymysql
from pymysql.err import IntegrityError, OperationalError
import os
from ultralytics import YOLOv10
from PIL import Image
import numpy as np
import supervision as sv
import cv2
import av, os
from PIL import Image
import shutil as sh

In [61]:
def annotate_image(input_img, label_annotator, bounding_box_annotator, model_pred, verbose = False):
    cont_img = np.ascontiguousarray(input_img, dtype=np.uint8)
    results = model_pred(cont_img, conf=0.10, verbose = verbose)[0]
    detections = sv.Detections.from_ultralytics(results)
    annotated_image = bounding_box_annotator.annotate(
        scene=input_img, detections=detections)
    annotated_image = label_annotator.annotate(
        scene=annotated_image, detections=detections)
    return(annotated_image)

def annotate_video(in_path, out_path = '.', model_file = 'best.pt', im_width = 416, im_height = 416):
    bba = sv.BoundingBoxAnnotator()
    la = sv.LabelAnnotator()
    model = YOLOv10(model_file)

    container = av.open(in_path)
    stream_vid = container.streams.video[0]
    fname = in_path.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    out_path = f'{out_path}/{fname[:per_index]}_annotated.mp4'
    outp = av.open(out_path, 'w')
    codec_name = stream_vid.codec_context.name
    fps = stream_vid.codec_context.rate
    output_stream = outp.add_stream(codec_name, str(fps))
    output_stream.width = im_width
    output_stream.height = im_height
    output_stream.pix_fmt = stream_vid.codec_context.pix_fmt
    for index, frame in enumerate(container.decode(stream_vid)):
        pil_img = frame.to_image()
        np_img = np.array(pil_img)
        np_img_resize = cv2.resize(np_img, (im_width, im_height))
        np_rot = np_img_resize[:, :, ::-1]
        small_pil_img = Image.fromarray(np_rot)
        np_image_2 = np.array(small_pil_img)
        an_mg = annotate_image(np_image_2, label_annotator = la, bounding_box_annotator=bba, model_pred = model, verbose = False)
        frame_out = av.VideoFrame.from_ndarray(an_mg, format='bgr24')
        pkt = output_stream.encode(frame_out)
        outp.mux(pkt)
    container.close()
    outp.close()

    print(f'Annotated video has been saved as {out_path}')
    return True

In [157]:
db_main = 'test_4'
cur_name = 'ab'


In [158]:
sql_conn = pymysql.connect(
    user="root",
    password="dbuserdbuser",
    host="localhost",
    port=3306,
    database=db_main,
    cursorclass=pymysql.cursors.DictCursor,
    autocommit=True)

cur = sql_conn.cursor()
res = cur.execute("SELECT 1")
res = cur.fetchall()
res

[{'1': 1}]

In [159]:
def add_user(name, password):
    '''
    Returns: 1 if query is successful, 0 if not
    '''

    try:
        res = cur.execute(f"INSERT INTO people (Username, Password, Time_Created) VALUES ('{name}', '{password}', CURRENT_TIMESTAMP );")
    except IntegrityError:
        print(f"Integrity Error Violated: Duplicate User with {name}")
        res = 0
    return res
add_user(cur_name, '123')

1

In [160]:
def get_ext(fpath):
    fname = fpath.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    ext = fname[per_index + 1:].lower()
    return ext

def get_id_fname(f_out, fpath, id):
    fname = fpath.rsplit('/', 1)[-1]
    per_index = fname.index('.')
    ext = fname[per_index + 1:].lower()
    return f"{f_out}/{fname[:per_index]}-{id}.{ext}"

def get_REPLACE_ID():
    cur.execute(f"""SELECT Raw_File_ID from raw_files where Filepath = 'REPLACE'""")
    id = cur.fetchall()[-1]['Raw_File_ID']
    return id

def add_photo(name, fpath, notes = '', f_out = './Files/Image_raw'):
    '''
    Returns: Index of added photo if successful, 0 if not
    '''

    im = Image.open(fpath)
    width = im.size[0]
    height = im.size[1]
    fsize = os.stat(fpath).st_size
    ext = get_ext(fpath)

    cur.execute(f"INSERT INTO raw_files (Username, Filepath, Size, Type, Extension, Notes, Width, Height, Time_Uploaded) VALUES ('{name}', 'REPLACE', {fsize}, 'Image', '{ext}', '{notes}', {width}, {height}, CURRENT_TIMESTAMP);")
    id = get_REPLACE_ID()
    f_id_name = get_id_fname(f_out, fpath, id)
    im.save(f_id_name)

    cur.execute(f"UPDATE raw_files SET Filepath = '{f_id_name}' WHERE Raw_File_ID = {id};")
    return id





id_raw_photo = add_photo(cur_name, fpath = 'C:/Users/micha/OneDrive/Desktop/Shellfish_project_2024/Jupyter_local_code/dummy.jpg', notes = 'after significant changes')

In [164]:
import cv2
def add_video(name, fpath, notes = '', f_out = './Files/Video_raw'):
    '''
    Returns: Index of added video if successful, 0 if not
    '''
    cap = cv2.VideoCapture(fpath)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fsize = os.stat(fpath).st_size
    ext = get_ext(fpath)
    
    cur.execute(f"INSERT INTO raw_files (Username, Filepath, Size, Type, Extension, Notes, Width, Height, Time_Uploaded) VALUES ('{name}', 'REPLACE', {fsize}, 'Video', '{ext}', '{notes}', {width}, {height}, CURRENT_TIMESTAMP);")
    id = get_REPLACE_ID()
    f_id_name = get_id_fname(f_out, fpath, id)
    sh.copyfile(fpath, f_id_name)

    cur.execute(f"UPDATE raw_files SET Filepath = '{f_id_name}' WHERE Raw_File_ID = {id};")

    fps = cap.get(cv2.CAP_PROP_FPS)
    color_order = 'RGB' # FIXME - cant figure out how to extract from cv2 object

    cur.execute(f"INSERT INTO videos (Raw_File_ID, FPS, Color_Order) VALUES ('{id}', '{fps}', '{color_order}');")

    return id

fpath_vid = "C:/Users/micha/OneDrive/Desktop/Shellfish_project_2024/GOPR1077.MP4"
id_raw_video = add_video(cur_name, fpath_vid, notes='without dummy files')

In [156]:
sh.copyfile(fpath_vid, "./Files/Video_raw/GOPR1077-17.mp4")

'./Files/Video_raw/GOPR1077-17.mp4'

In [57]:
b = add_user('ab', 'cde')
print(b)

Integrity Error Violated: Duplicate Primary Key with ab
False


In [46]:

res = cur.execute("DELETE FROM test_db.people WHERE Username = 'test_man';")
res = cur.execute("INSERT INTO test_db.people (Username, Password, Time_Created) VALUES ('test_man', 'test_pwd_2', CURRENT_TIMESTAMP );")
